<a href="https://colab.research.google.com/github/saotomryo/Use_MMOCR/blob/main/Use_MMOCR_fcenet_r50_fpn_1500e_icdar2015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import json
from skimage import measure
import numpy as np
print(torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device = ", device)

## 環境準備 MMCV MMOCRのインストール

In [ ]:
#!pip install mmcv-full
!pip install https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/mmcv_full-1.4.7-cp37-cp37m-manylinux1_x86_64.whl

In [ ]:
# Install mmdetection
!pip install mmdet

In [ ]:
HOME_PATH = "/content"

In [ ]:

!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements/build.txt
!pip install -v -e .  # or "python setup.py develop"

## コンフィグファイルの編集

In [ ]:
%cd /content/mmocr

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/content/mmocr/configs/textdet/fcenet/fcenet_r50_fpn_1500e_icdar2015.py')


## アノテーションフォルダの設定

　以下の形式で作成したJSONファイルが必要です。
 基本的には、 COCO Datasetと同様の形式になりますが、対象の文字列を記載したテキストファイルのファイル名を入れる必要があります。
対象のテキストファイルは、ルートパスの下に"annotation"というフォルダを作成して、annotationフォルダから下のファイルパスを記載してください。

- 
instances_test.json
```   
{  
    "images":
        ["file_name":画像ファイル名,  
        "height":画像の高さ,  
        "width":画像の横幅,  
        "segm_file":文字列を保存したテキストファイルのファイル名,
        "id",画像のファイルのID(一意）],   
        [.....],  
        ....,  
    "categories":
        [{"id": 1, "name": "text"}],  
    "annotations":  
        [{"iscrowd": 1(たくさんの文字があれば１),   
        "category_id": "1"(基本的にはテキスト以外のものは対象がなので、常に1),   
        "bbox": [左上のx座標, 左上のy座標, 高さ, 幅],   
        "area": 面積（bboxから計算可能),  
        "segmentation": [[x1, y1, x2, y2, x3 , y3 , x4 , y4 , .......]](ポリゴン形式で各頂点の座標を左周りに記入),   
        "image_id": 画像のファイルID,   
        "id": アノテーションのID(一意)}, 
        {....},  
        ....
    ]  
}

```

In [ ]:
## アノテーションフォルダの設定


train_label_path = 訓練用のJSONファイル
val_label_path = 検証用のJSONファイル

# 画像ファイルのパス
train_img_path = 訓練用画像ファイルのフォルダパス
val_img_path = 検証用画像ファイルのフォルダパス

# ルートフォルダ
root_path = ルートフォルダのパス


# 作業フォルダ（学習時のモデル出力先（エポック毎に出力))
work_path = '/content/work_dir'

In [ ]:
from mmdet.apis import set_random_seed

# コンフィグを変更する
# fcenet

# 作業フォルダ
cfg.work_dir = work_path

# rootフォルダ
cfg.data_root = root_path

# アノテーションファイルのパス
cfg.train.ann_file = train_label_path
cfg.train.img_prefix = train_img_path
cfg.test.ann_file = val_img_path
cfg.test.img_prefix = val_img_path
cfg.train_list[0].ann_file = train_label_path
cfg.train_list[0].img_prefix = train_img_path
cfg.test_list[0].ann_file = val_label_path
cfg.test_list[0].img_prefix = val_img_path
cfg.data.train.datasets[0].ann_file = train_label_path
cfg.data.train.datasets[0].img_prefix = train_img_path
cfg.data.test.datasets[0].ann_file = val_label_path
cfg.data.test.datasets[0].img_prefix = val_img_path
cfg.data.val.datasets[0].ann_file  = val_label_path
cfg.data.val.datasets[0].img_prefix = val_img_path

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.001 / 8
cfg.lr_config.warmup = None
# Choose to log training results every 40 images to reduce the size of log file. 
cfg.log_config.interval = 40

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let's have a look at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


# 編集したコンフィグをファイル出力
cfg.dump('/content/fcenet_r50_fpn_1500e_icdar2015.py')

In [ ]:
from mmocr.datasets import build_dataset
from mmocr.models import build_detector
from mmocr.apis import train_detector
from mmocr.apis import init_detector
import os.path as osp

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

#追加学習をする場合
#checkpoint = 学習済みモデルのファイルパス
#model = init_detector(cfg, checkpoint, device=device)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
#mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

## 学習結果の確認

In [ ]:
%cd /content/mmocr

In [ ]:

from mmocr.apis import init_detector, model_inference

img = #確認する画像ファイルのパス
checkpoint = # 利用するモデルのパス
out_file = # 出力するファイルのパス

model = init_detector(cfg, checkpoint, device=device)
if model.cfg.data.test['type'] == 'ConcatDataset':
    model.cfg.data.test.pipeline = model.cfg.data.test['datasets'][0].pipeline

result = model_inference(model, img)
print(f'result: {result}')

